# Método dos Nós

O **Método dos Nós** é uma técnica da engenharia estrutural para determinar as forças internas em cada membro de uma treliça. O princípio é simples: se a estrutura inteira está em equilíbrio, então cada um de seus nós (as juntas onde os membros se conectam) também deve estar em equilíbrio.

Isso significa que, para cada nó, a soma de todas as forças que atuam sobre ele deve ser zero. Descompondo isso nos eixos cartesianos, temos as duas equações de equilíbrio que são a base do método:

Soma das forças na direção horizontal (X) é zero: 
$$∑Fx = 0$$

Soma das forças na direção vertical (Y) é zero: 
$$∑Fy = 0$$

Como temos duas equações para cada nó, só podemos resolver o sistema se tivermos, no máximo, duas forças desconhecidas.

## Passo 1: Preparação e Estruturas de Dados

In [ ]:
# Dicionario de forças

external_forces = {point: (0.0, 0.0) for point in self.points}

In [ ]:
# Dicionario de conexões por junta

joint_connections = {point: [] for point in self.points}

for line in self.lines:
    joint_connections[line.start].append(line)
    joint_connections[line.end].append(line)

Este é um **mapa de conectividade**. Ele itera sobre todas as linhas da ponte e, para cada, adiciona-a à lista de conexões de seu ponto inicial e final. Ao final, teremos um dicionário onde cada ponto está associado a uma lista de todos os membros que se conectam a ele.

Quem sabe o **mapa de conectividade** não poderia ser parte da classe `Truss`?

In [ ]:
# Reset das forças nas linhas

for line in self.lines:
    line.force_magnitude = None

## Passo 2: Identificação e Aplicação das Forças Externas

In [ ]:
# Aplicação da carga externa no ponto de carga

if self.load_point:
    external_forces[self.load_point] = (0.0, -self.load)

In [ ]:
# Reação nos apoios

for point, reaction in self.support_reactions.items():
    fx, fy = external_forces[point]
    external_forces[point] = (fx + reaction[0], fy + reaction[1])

As reações nos apoios, pré-calculadas no `__init__` da classe Bridge, são adicionadas. Essas reações são forças verticais que o solo exerce sobre a ponte para sustentá-la. No nosso caso, como a carga é central, elas valem $load / 2$ em cada apoio.

Ao final desta etapa, o dicionário `external_forces` contém um panorama completo de todas as forças externas que atuam na estrutura.

## Passo 3: O Loop de Resolução Iterativa

In [ ]:
unsolved_joints = list(self.points)
progress_made = True
while unsolved_joints and progress_made:
    progress_made = False
    for joint in list(unsolved_joints):
        # lógica de resolução
        pass

`unsolved_joints`: Uma lista com todos os nós da ponte é criada. À medida que um nó é resolvido, ele é removido desta lista. O loop continua até que a lista esteja vazia.

`progress_made`: Esta é uma variável de controle. Se em uma iteração completa do loop for nenhum nó puder ser resolvido, significa que a estrutura é **estaticamente indeterminada** (tem mais incógnitas do que equações) ou instável, e o loop é interrompido para evitar um ciclo infinito.

O `for joint in list(unsolved_joints)` procura em cada iteração por um nó que atenda à condição fundamental: ter no máximo duas forças de membros desconhecidas. Normalmente, os primeiros nós a serem resolvidos são os dos apoios.

## Passo 4: A Análise do Nó

In [ ]:
# Quando um nó é solucionável

sum_fx = external_forces[joint][0]
sum_fy = external_forces[joint][1]

for line in joint_connections[joint]:
    if line.force_magnitude is not None:
        # ... adiciona componentes x e y da força conhecida ...

Primeiro, as componentes `sum_fx` e `sum_fy` são inicializadas com as forças externas no nó. Em seguida, o código itera sobre todos os membros conectados. Se a força de um membro já foi calculada, suas componentes X e Y são decompostas e somadas a `sum_fx` e `sum_fy`.

### Montagem do Sistema de Equações

O código agora foca apenas nos 2 (ou 1) membros com forças desconhecidas. Para cada membro desconhecido, ele calcula o ângulo que o membro faz com a horizontal, usando math.atan2. Este ângulo é usado para encontrar os cossenos e senos, que são os coeficientes das nossas equações de equilíbrio.

Assumindo duas forças desconhecidas, F1 (na line1) e F2 (na line2), as equações de equilíbrio são:

$$(∑F_{x_conhecidas})+F_{1} cos(θ_{1})+F_{2} cos(θ_{2})=0$$

$$(∑F_{y_conhecidas})+F_{1} sin(θ_{1})+F_{2} sin(θ_{2})=0$$

Por convenção, assumimos que as forças desconhecidas são de tração, ou seja, estão "saindo" do nó. Se o resultado final for negativo, significa que a força é, na verdade, de compressão.

Para resolver o sistema, rearranjamos as equações para a forma matricial A⋅x=B:

$$
\begin{bmatrix}
\cos(\theta_1) & \cos(\theta_2) \\
\sin(\theta_1) & \sin(\theta_2)
\end{bmatrix}
\begin{bmatrix}
F_1 \\
F_2
\end{bmatrix}
=
\begin{bmatrix}
-\sum F_{x_{\text{conhecidas}}} \\
-\sum F_{y_{\text{conhecidas}}}
\end{bmatrix}
$$

Onde:
- A é a matriz dos coeficientes (os cossenos e senos dos ângulos).
- x é o vetor das forças incógnitas que queremos descobrir ($F_1$ e $F_2$).
- B é o vetor resultante das forças já conhecidas (com o sinal invertido, pois as passamos para o outro lado da equação).

In [ ]:
# Construção da matriz com numpy

A = np.array([
    [math.cos(angle1), math.cos(angle2)],
    [math.sin(angle1), math.sin(angle2)]
])

B = np.array([-sum_fx, -sum_fy])


# Resolução do sistema de equações
forces = np.linalg.solve(A, B)

### **Caso especial**: Apenas uma força desconhecida

Para cada nó, temos duas equações fundamentais para garantir o equilíbrio:

Soma das forças em X é zero ($∑F_x = 0$).

Soma das forças em Y é zero ($∑F_y = 0$).

O problema aqui é que temos duas equações, mas apenas uma incógnita (a força $F_1$ no único membro desconhecido). Matematicamente, isso é um sistema **super-restringido** ou **sobredeterminado**.

---

Ter um sistema **super-restringido** significa que não temos liberdade para simplesmente "escolher" um valor para a nossa única incógnita, $F_1$, que satisfaça as duas equações. Para que o nó esteja realmente em equilíbrio, o valor de $F_1$ que satisfaz a equação de forças em X deve ser exatamente o mesmo valor de $F_1$ que satisfaz a equação de forças em Y. 

Se as duas equações resultarem em valores diferentes para $F_1$, isso indica um desequilíbrio. Em uma estrutura real, isso significaria que o nó não está parado — a estrutura estaria instável ou em movimento. No nosso algoritmo, isso apontaria para um erro de cálculo anterior ou uma estrutura mal definida.

---

Resolver para uma direção (Calcular): O código não tenta resolver as duas equações ao mesmo tempo. Ele usa apenas uma delas para calcular um valor candidato para a força $F_1$.

Primeiro, tentamos usar a equação das forças em X:

$$F_1 \cos(θ_1) = -∑F_{x_{conhecidas}}$$
$$F_1 = \frac{-∑F_{x_{conhecidas}}}{\cos(θ_1)}$$

Isso é o que a linha `force = -sum_fx / math.cos(angle1)` faz.

Depois de encontrar o valor de `force`, o código o utiliza como prova. Inserimos esse valor na segunda equação para ver se ela "fecha", ou seja, se o resultado é zero (dentro de uma pequena margem de erro para cálculos de ponto flutuante).

A verificação é: O valor de `force * math.sin(angle1) + sum_fy` é próximo de zero?

In [ ]:
# Resolve para uma única força

# Pode estar super-restringido, mas resolvemos para uma direção e verificamos a outra
if abs(math.cos(angle1)) > 1e-9: # Evita divisão por zero
    force = -sum_fx / math.cos(angle1)

    # Verificação de equilíbrio na outra direção
    if abs(force * math.sin(angle1) + sum_fy) > 1e-6:
        print(f"Warning: Force imbalance at joint ({joint.x}, {joint.y}) with one unknown.")

elif abs(math.sin(angle1)) > 1e-9:
    force = -sum_fy / math.sin(angle1)

    # Verificação de equilíbrio na outra direção
    if abs(force * math.cos(angle1) + sum_fx) > 1e-6:
        print(f"Warning: Force imbalance at joint ({joint.x}, {joint.y}) with one unknown.")

else:
    force = 0 # Membro de força zero

line1.force_magnitude = force

## Passo 5: Armazenamento e Repetição

In [ ]:
# Armazenamento das forças calculadas
line1.force_magnitude = forces[0]
line2.force_magnitude = forces[1]

# Remoção do nó resolvido da lista de nós não resolvidos
unsolved_joints.remove(joint)
progress_made = True

Este ciclo se repete. Com as forças de `line1` e `line2` agora conhecidas, outros nós vizinhos que antes tinham 3 ou mais incógnitas podem agora ter apenas 2, tornando-se "solucionáveis" na próxima iteração do loop. O processo continua até que todos os membros da treliça tenham suas forças internas determinadas.

## Passo 6: Interpretação dos Resultados

A convenção de sinais adotada para o resultado final é um padrão na engenharia estrutural:

- Força Positiva (> 0): Indica **Tração**. O membro está sendo esticado, ou "puxado" pelas suas extremidades. Na imagem, representamos em verde.

- Força Negativa (< 0): Indica **Compressão**. O membro está sendo "esmagado", ou empurrado em suas extremidades. Na imagem, representamos em vermelho.

- Força Próxima de Zero: Indica um **membro de força nula**, que não está sob carga naquela condição específica, mas pode ser essencial para a estabilidade da estrutura.